<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/Classify_Twilight_for_Nocturnal_Flight_Calls_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Tools

In [1]:
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

# Load Source

## Select NFC's

In [14]:
# 2012_to_2020NFCBandingData_20210106.csv
src_nfc = "https://drive.google.com/uc?id=1r2esEc-UdAqoBvptlELey6tbfGmvHbGI"

In [55]:
df_nfc = pd.read_csv(src_nfc, sep=',')

In [21]:
df_nfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63599 entries, 0 to 63598
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   season                   63599 non-null  object 
 1   year                     63599 non-null  int64  
 2   detector                 63599 non-null  object 
 3   species                  63599 non-null  object 
 4   site                     63599 non-null  object 
 5   date                     63599 non-null  object 
 6   recording_start          63599 non-null  object 
 7   recording_length         63599 non-null  object 
 8   elapsed_detection_time   63599 non-null  object 
 9   real_detection_time      63599 non-null  object 
 10  real_detection_datetime  63599 non-null  object 
 11  rounded_to_half_hour     63599 non-null  object 
 12  duplicate                62568 non-null  object 
 13  sunset                   63599 non-null  object 
 14  civil_dusk            

In [56]:
df_nfc.head(n=3)

,season,year,detector,species,site,date,recording_start,recording_length,elapsed_detection_time,real_detection_time,real_detection_datetime,rounded_to_half_hour,duplicate,sunset,civil_dusk,nautical_dusk,astronomical_dusk,astronomical_dawn,nautical_dawn,civil_dawn,sunrise,moon_altitude,moon_illumination
0,Fall,2015,thrush,amco,flood,8/23/2015,20:03:00,6:45:47,4:19:13,0:22:13,8/24/2015 0:22,0:30:00,no,8/23/2015 20:32,8/23/2015 21:04,8/23/2015 21:43,8/23/2015 22:26,8/24/2015 4:51,8/24/2015 5:33,8/24/2015 6:13,8/24/2015 6:45,7.0,64.8
1,Fall,2015,thrush,amco,flood,8/23/2015,2:48:46,4:26:16,0:18:59,3:07:44,8/24/2015 3:07,3:00:00,no,8/23/2015 20:32,8/23/2015 21:04,8/23/2015 21:43,8/23/2015 22:26,8/24/2015 4:51,8/24/2015 5:33,8/24/2015 6:13,8/24/2015 6:45,-18.1,65.9
2,Fall,2015,thrush,amco,flood,9/15/2015,2:03:46,5:41:16,3:08:40,5:12:26,9/16/2015 5:12,5:00:00,no,9/15/2015 19:48,9/15/2015 20:18,9/15/2015 20:54,9/15/2015 21:31,9/16/2015 5:31,9/16/2015 6:08,9/16/2015 6:44,9/16/2015 7:15,-47.8,8.9


# Wrangle

## Date Format

In [57]:
df_nfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63599 entries, 0 to 63598
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   season                   63599 non-null  object 
 1   year                     63599 non-null  int64  
 2   detector                 63599 non-null  object 
 3   species                  63599 non-null  object 
 4   site                     63599 non-null  object 
 5   date                     63599 non-null  object 
 6   recording_start          63599 non-null  object 
 7   recording_length         63599 non-null  object 
 8   elapsed_detection_time   63599 non-null  object 
 9   real_detection_time      63599 non-null  object 
 10  real_detection_datetime  63599 non-null  object 
 11  rounded_to_half_hour     63599 non-null  object 
 12  duplicate                62568 non-null  object 
 13  sunset                   63599 non-null  object 
 14  civil_dusk            

In [58]:
date_columns = ['real_detection_datetime', 'sunset', 'civil_dusk', 'nautical_dusk', 'astronomical_dusk',\
          'astronomical_dawn', 'nautical_dawn', 'civil_dawn', 'sunrise']

In [59]:
for column in date_columns:
    df_nfc[column] = pd.to_datetime(df_nfc.loc[:, column]).dt.tz_localize('US/Mountain')

In [60]:
df_nfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63599 entries, 0 to 63598
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype                      
---  ------                   --------------  -----                      
 0   season                   63599 non-null  object                     
 1   year                     63599 non-null  int64                      
 2   detector                 63599 non-null  object                     
 3   species                  63599 non-null  object                     
 4   site                     63599 non-null  object                     
 5   date                     63599 non-null  object                     
 6   recording_start          63599 non-null  object                     
 7   recording_length         63599 non-null  object                     
 8   elapsed_detection_time   63599 non-null  object                     
 9   real_detection_time      63599 non-null  object                     
 10

## Classify

In [61]:
def dusk_dawn(row):
    detection = row['real_detection_datetime']
    
    if row['sunset'] <= detection < row['civil_dusk']:
        return 'civil_dusk'
    elif row['civil_dusk'] <= detection < row['nautical_dusk']:
        return 'nautical_dusk'
    elif row['nautical_dusk'] <= detection < row['astronomical_dusk']:
        return 'astronomical_dusk'
    elif row['astronomical_dusk'] <= detection < row['astronomical_dawn']:
        return 'night'
    elif row['astronomical_dawn'] <= detection < row['nautical_dawn']:
        return 'astronomical_dawn'
    elif row['nautical_dawn'] <= detection < row['civil_dawn']:
        return 'nautical_dawn'
    elif row['civil_dawn'] <= detection < row['sunrise']:
        return 'civil_dawn'
    else:
        print('Row out of range')

In [62]:
df_nfc['light_classification'] = df_nfc.apply(dusk_dawn, axis=1)

In [67]:
df_nfc.head()

,season,year,detector,species,site,date,recording_start,recording_length,elapsed_detection_time,real_detection_time,real_detection_datetime,rounded_to_half_hour,duplicate,sunset,civil_dusk,nautical_dusk,astronomical_dusk,astronomical_dawn,nautical_dawn,civil_dawn,sunrise,moon_altitude,moon_illumination,light_classification
0,Fall,2015,thrush,amco,flood,8/23/2015,20:03:00,6:45:47,4:19:13,0:22:13,2015-08-24 00:22:00-06:00,0:30:00,no,2015-08-23 20:32:00-06:00,2015-08-23 21:04:00-06:00,2015-08-23 21:43:00-06:00,2015-08-23 22:26:00-06:00,2015-08-24 04:51:00-06:00,2015-08-24 05:33:00-06:00,2015-08-24 06:13:00-06:00,2015-08-24 06:45:00-06:00,7.0,64.8,night
1,Fall,2015,thrush,amco,flood,8/23/2015,2:48:46,4:26:16,0:18:59,3:07:44,2015-08-24 03:07:00-06:00,3:00:00,no,2015-08-23 20:32:00-06:00,2015-08-23 21:04:00-06:00,2015-08-23 21:43:00-06:00,2015-08-23 22:26:00-06:00,2015-08-24 04:51:00-06:00,2015-08-24 05:33:00-06:00,2015-08-24 06:13:00-06:00,2015-08-24 06:45:00-06:00,-18.1,65.9,night
2,Fall,2015,thrush,amco,flood,9/15/2015,2:03:46,5:41:16,3:08:40,5:12:26,2015-09-16 05:12:00-06:00,5:00:00,no,2015-09-15 19:48:00-06:00,2015-09-15 20:18:00-06:00,2015-09-15 20:54:00-06:00,2015-09-15 21:31:00-06:00,2015-09-16 05:31:00-06:00,2015-09-16 06:08:00-06:00,2015-09-16 06:44:00-06:00,2015-09-16 07:15:00-06:00,-47.8,8.9,night
3,Fall,2015,thrush,amco,flood,9/15/2015,2:03:46,5:41:16,3:08:41,5:12:26,2015-09-16 05:12:00-06:00,5:00:00,yes,2015-09-15 19:48:00-06:00,2015-09-15 20:18:00-06:00,2015-09-15 20:54:00-06:00,2015-09-15 21:31:00-06:00,2015-09-16 05:31:00-06:00,2015-09-16 06:08:00-06:00,2015-09-16 06:44:00-06:00,2015-09-16 07:15:00-06:00,-47.8,8.9,night
4,Fall,2016,thrush,amco,flood,8/28/2016,19:52:00,6:45:47,5:45:28,1:37:28,2016-08-29 01:37:00-06:00,1:30:00,no,2016-08-28 20:21:00-06:00,2016-08-28 20:53:00-06:00,2016-08-28 21:31:00-06:00,2016-08-28 22:12:00-06:00,2016-08-29 05:01:00-06:00,2016-08-29 05:42:00-06:00,2016-08-29 06:21:00-06:00,2016-08-29 06:52:00-06:00,-19.2,10.0,night


In [71]:
# Look for unclassified rows
df_nfc.loc[df_nfc['light_classification'] == 'Row out of range']

,season,year,detector,species,site,date,recording_start,recording_length,elapsed_detection_time,real_detection_time,real_detection_datetime,rounded_to_half_hour,duplicate,sunset,civil_dusk,nautical_dusk,astronomical_dusk,astronomical_dawn,nautical_dawn,civil_dawn,sunrise,moon_altitude,moon_illumination,light_classification


# Output

In [70]:
output_filename = '2012_to_2020NFCBandingData_20210106_lightclassified.csv'
df_nfc.to_csv(output_filename, index=False)

# Note

These classifications assume that the twilight datetimes provided are appropriate for the lat long of the station.